In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta, time, date
import re

import xlsxwriter

## Input and preprocessing

In [2]:
# define the path of input files -> rbd=raw billing data, ra=route analysis, ba=BA raw report
path_rbd= 'D:/solutioning/Morgan Stanley Blr/RBD.csv'
path_ba= 'D:/solutioning/Morgan Stanley Blr/BA.csv'
path_ra= 'D:/solutioning/Morgan Stanley Blr/RA.csv'

# Converting cab type to seating capacity. If capacity can be inferred from cab type column leave it as '{}'
rbd_cab_capacity_mapping = {}
ra_cab_capacity_mapping = {'Toyota Etios':4, 'Toyota Innova':6, 'StandBy':4}

# define the start and end date for the seat utilization analysis
seat_util_start_date = datetime.strptime("03-02-2020", "%d-%m-%Y")
seat_util_end_date = datetime.strptime("07-02-2020", "%d-%m-%Y")

# start and end date to analyse the commercials deployed against vendor and contract
commercial_start_date = datetime.strptime("03-02-2020", "%d-%m-%Y")
commercial_end_date = datetime.strptime("07-02-2020", "%d-%m-%Y")

# based on trip status, which all trips are to be included in the analysis of fleet mix and commercials
trips_to_include = ['Trip Completed', 'Trip Marked Noshow', 'Trip in Progress', 'Trip cancelled from dashboard']

# define where the result file needs to be stored
output_path = 'D:/solutioning/Morgan Stanley Blr/result.xlsx'

###### RBD

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
# df_rbd_jan = pd.read_csv("D:/solutioning/Office/RBD_Jan.csv", infer_datetime_format=True)
# df_rbd_feb = pd.read_csv("D:/solutioning/infosys-iblr/RBD_Feb.csv", infer_datetime_format=True)
# df_rbd_mar = pd.read_csv("D:/solutioning/Office/RBD_Mar.csv", infer_datetime_format=True)
# df_rbd_apr = pd.read_csv("D:/solutioning/Office/RBD_Apr.csv", infer_datetime_format=True)
# df_rbd_may = pd.read_csv("D:/solutioning/Office/RBD_May.csv", infer_datetime_format=True)
# df_rbd_jun = pd.read_csv("D:/solutioning/Office/RBD_Jun.csv", infer_datetime_format=True)
# df_rbd_jul = pd.read_csv("D:/solutioning/Genpact-Hyd/RBD_Jul.csv", infer_datetime_format=True)
# df_rbd_aug = pd.read_csv("D:/solutioning/Office/RBD_Aug.csv", infer_datetime_format=True)
# df_rbd_sep = pd.read_csv("D:/solutioning/Office/RBD_Sep.csv", infer_datetime_format=True)
# df_rbd_oct = pd.read_csv("D:/solutioning/Wipro-SJP/RBD_Oct.csv", infer_datetime_format=True)
# df_rbd_nov = pd.read_csv("D:/solutioning/Office/RBD_Nov.csv", infer_datetime_format=True)
# df_rbd_dec = pd.read_csv("D:/solutioning/Office/RBD_Dec.csv", infer_datetime_format=True)

# df_rbd = pd.concat([df_rbd_jul], ignore_index=True, sort=False)
# del [df_rbd_jul]

In [5]:
df_rbd = pd.read_csv(path_rbd, infer_datetime_format=True)

In [6]:
def date1(x):
    if x=='':
        return np.nan
    else:
        format = '%d-%b-%y'
        return datetime.strptime(x, format)

def datentime1(x):
    if x=='':
        return x
    else:
        if len(x)>18:
            format = '%d-%b-%y %H:%M:%S'
            return datetime.strptime(x, format)
        elif len(x)<18:
            format = '%d-%b-%y %H:%M'
            return datetime.strptime(x, format)

In [7]:
# change the data type of Leg date, duty start and duty end. 

df_rbd['Leg Date'] = df_rbd['Leg Date'].replace(np.nan, '').apply(date1)

for x in ['Duty Start', 'Duty End']:
    df_rbd[x] = df_rbd[x].apply(datentime1)

In [8]:
# add column for cab capcity 
if len(rbd_cab_capacity_mapping) == 0:
    df_rbd['Cab Capacity'] = df_rbd['Contract'].replace(np.nan,"").apply(lambda x: int(re.findall("\d+", 
                                                                                            x)[0]) if len(x)>0 else None)
else:
    df_rbd['Cab Capacity'] = df_rbd['Contract'].replace(np.nan,'').apply(lambda x: rbd_cab_capacity_mapping[x] if len(x)>0 else None)                                                                      

In [9]:
# add a weekday column to the table

def day_of_week(x):
    y = {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    if x!=x:            #null values are not equal to themselves
        return ''
    else:
        return y[x.weekday()]

df_rbd['Weekday'] = df_rbd['Leg Date'].apply(day_of_week)

# excluding weekends

# df_rbd_wkday = df_rbd.dropna(subset=['Leg Date'])
# df_rbd_wkday = df_rbd_wkday[df_rbd_wkday['Weekday'].apply(lambda x: True if x in ['Mon', 'Tue', 
#                                                                                        'Wed', 'Thu', 'Fri'] else False)]
# df_rbd = df_rbd_wkday

###### BA Report

In [162]:
# df_ba_jan = pd.read_csv("D:/solutioning/anushka/iqvia/Nov BAR.csv", infer_datetime_format=True)
# df_ba_feb = pd.read_csv("D:/solutioning/infosys-iblr/BA_Feb.csv", infer_datetime_format=True)
# df_ba_mar = pd.read_csv("D:/solutioning/Office/BA_Mar.csv", infer_datetime_format=True)
# df_ba_apr = pd.read_csv("D:/solutioning/Office/BA_Apr.csv", infer_datetime_format=True)
# df_ba_may = pd.read_csv("D:/solutioning/Office/BA_May.csv", infer_datetime_format=True)
# df_ba_jun = pd.read_csv("D:/solutioning/Office/BA_Jun.csv", infer_datetime_format=True)
# df_ba_jul = pd.read_csv("D:/solutioning/Office/BA_Jul.csv", infer_datetime_format=True)
# df_ba_aug = pd.read_csv("D:/solutioning/Office/BA_Aug.csv", infer_datetime_format=True)
# df_ba_sep = pd.read_csv("D:/solutioning/Office/BA_Sep.csv", infer_datetime_format=True)
# df_ba_oct = pd.read_csv("D:/solutioning/Wipro-SJP/BA_Oct.csv", infer_datetime_format=True)
# df_ba_nov = pd.read_csv("D:/solutioning/Office/BA_Nov.csv", infer_datetime_format=True)
# df_ba_dec = pd.read_csv("D:/solutioning/Office/BA_Dec.csv", infer_datetime_format=True)

# df_ba = pd.concat([df_ba_feb], ignore_index=True, sort=False)
# del [df_ba_feb]

In [163]:
df_ba = pd.read_csv(path_ba, infer_datetime_format=True)

In [164]:
def date(x):
    format = '%b %d, %Y'
    return datetime.strptime(x, format)

def datentime(x):
    format = '%b %d,%Y %H:%M'
    if x=='':
        return x
    else: 
        return datetime.strptime(x, format)

In [165]:
# change the date and the five datetimes in the report to datetime

df_ba['Date'] = df_ba['Date'].apply(date)

for x in ['Planned Pickup Time', 'Driver Report Time', 'Actual Pickup Time', 'Planned Drop Time', 'Actual Drop Time']:
    df_ba[x] = df_ba[x].replace('--', '').apply(datentime)

In [166]:
# add a weekday column to the table

df_ba['Weekday'] = df_ba['Date'].apply(day_of_week)

In [167]:
# concatenate direction and shift time into a column named 'Shift'

df_ba['Shift'] = df_ba['Direction'].str.cat(df_ba['Shift Type/Time'], sep=' ')

##### RA Report

In [171]:
# df_ra_jan = pd.read_csv("D:/solutioning/Morgan Stanley Mum/RA_Nirlon.csv", infer_datetime_format=True)
# df_ra_feb = pd.read_csv("D:/solutioning/Morgan Stanley Mum/RA_GKC.csv", infer_datetime_format=True)
# df_ra_mar = pd.read_csv("D:/solutioning/Morgan Stanley Mum/RA_WeWork.csv", infer_datetime_format=True)
# df_ra_apr = pd.read_csv("D:/solutioning/Office/RA_Apr.csv", infer_datetime_format=True)
# df_ra_may = pd.read_csv("D:/solutioning/Office/RA_May.csv", infer_datetime_format=True)
# df_ra_jun = pd.read_csv("D:/solutioning/Office/RA_Jun.csv", infer_datetime_format=True)
# df_ra_jul = pd.read_csv("D:/solutioning/Office/RA_Jul.csv", infer_datetime_format=True)
# df_ra_aug = pd.read_csv("D:/solutioning/Office/RA_Aug.csv", infer_datetime_format=True)
# df_ra_sep = pd.read_csv("D:/solutioning/Office/RA_Sep.csv", infer_datetime_format=True)
# df_ra_oct = pd.read_csv("D:/solutioning/Wipro-SJP/RA_Oct.csv", infer_datetime_format=True)
# df_ra_nov = pd.read_csv("D:/solutioning/Office/RA_Nov.csv", infer_datetime_format=True)
# df_ra_dec = pd.read_csv("D:/solutioning/Office/RA_Dec.csv", infer_datetime_format=True)

# df_ra = pd.concat([df_ra_feb, df_ra_jan, df_ra_mar], ignore_index=True, sort=False)
# del [df_ra_feb, df_ra_jan, df_ra_mar]

In [172]:
df_ra = pd.read_csv(path_ra, infer_datetime_format=True)

In [173]:
# adding the cab capacity column
if len(ra_cab_capacity_mapping) == 0:
    df_ra['Cab Capacity'] = df_ra['Cab Type'].apply(lambda x: int(re.findall("\d+", x)[0]))
else:
    df_ra['Cab Capacity'] = df_ra['Cab Type'].apply(lambda x: ra_cab_capacity_mapping[x])

## Fleet Mix

##### Seat Utilization

In [174]:
# consider all trips where planned trip employee is not 0
temp = df_rbd[(df_rbd['Planned Trip Employees']!=0) & (df_rbd['Leg Type']=='Trip')]

In [175]:
# SU1 = sum of planned trip employees

SU1 = temp[(temp['Leg Date']>=seat_util_start_date) & (temp['Leg Date']<=seat_util_end_date)].pivot_table(
        values='Planned Trip Employees', index=['Leg Date', 'Trip Type'], columns='Cab Capacity', aggfunc='sum')

# SU2 = distinct count of trip ids

SU2 = temp[(temp['Leg Date']>=seat_util_start_date) & (temp['Leg Date']<=seat_util_end_date)].pivot_table(
        values='Trip Id', index=['Leg Date', 'Trip Type'], columns='Cab Capacity', aggfunc='count')
SU = SU1/SU2

# finding the seat utilization percentage

for i in list(np.arange(len((SU).columns))):
    SU.iloc[:,i] = SU.iloc[:,i]/SU.columns[i]*100

# concatenating all the table i.e #trips, #employees and seat utilization percentage.

temp_su = pd.concat([SU1, SU2, SU], axis=1, copy=False, keys=['employees', 'trips', 'seat util %'])

###### Deviation

In [176]:
# creating a concatenated table containing capacity, employees travelling and the max deviation for the trip against route ID

temp = pd.concat([df_ra.groupby('RouteId')['EmpID'].count(), df_ra.groupby('RouteId')['Extra distance'].max(),
                   df_ra.groupby('RouteId')['Cab Capacity'].mean()], axis=1, copy=False)
temp.rename(columns={'EmpID':'#employees'}, inplace=True, copy=False)
temp.reset_index(inplace=True)

# final tabular format of the information

temp = temp.groupby(['Cab Capacity', '#employees'])['Extra distance'].mean()
temp_dev = temp.unstack()

###### Fleet mix

In [177]:
# chopping dataframe containing only required trips based on the trip status

temp = df_rbd[df_rbd['Trip Status'].apply(lambda x: True if x in trips_to_include else False)]

# concatenating the 2 tables containing #unique cabs and %of the overall trips done cab capacity wise

temp_fm = pd.concat([temp.groupby('Cab Capacity')['Registration'].nunique(), 
                  ((temp.groupby('Cab Capacity')['Trip Id'].count()/temp['Trip Id'].count())*100)], axis=1, 
                    copy=False).rename(columns={'Registration':'#unique cabs', 'Trip Id':'%of trips done'})

## Bill Model

###### Trip Distances

In [178]:
# user defined slabs
user_defined_km_slabs = [10,15,20,25,30,40,50,60,150]

# taking completed trips

temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']
# inserting 0 in slabs to create bins for classification
user_defined_km_slabs.insert(0, 0)

# find number of trips by using cut function bins and applying value_counts to it

temp = pd.concat([pd.DataFrame(user_defined_km_slabs), pd.cut(temp['Leg Distance'], bins=user_defined_km_slabs, 
                                     right=False).value_counts().sort_index().reset_index(drop=True)], axis=1, copy=False)

# working on the presentation side

temp.drop(columns=[0], axis=1, inplace=True)
user_defined_km_slabs.remove(0)
temp = temp.dropna()
temp.set_axis(list(map(lambda x: str(x)+' km bin', user_defined_km_slabs)), inplace=True)
temp_td = temp.rename(columns={'Leg Distance':'# of trips'})

In [30]:
# user defined slabs
user_defined_km_slabs = [10,15,20,25,30,40,50,60,150]

# taking completed trips

temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']
# inserting 0 in slabs to create bins for classification
user_defined_km_slabs.insert(0, 0)

# find number of trips by using cut function bins and applying value_counts to it

temp = pd.concat([pd.cut(temp['Leg Distance'], bins=user_defined_km_slabs, 
                         right=False).rename('km brackets'), temp], axis=1, copy=False)
#temp = pd.concat([pd.DataFrame(user_defined_km_slabs), pd.cut(temp['Leg Distance'], bins=user_defined_km_slabs, 
#                                     right=False).value_counts().sort_index()], axis=1, copy=False)

temp = temp.groupby('Cab Capacity')['km brackets'].value_counts().sort_index()
temp.unstack(level=0)
# working on the presentation side

# temp.drop(columns=[0], axis=1, inplace=True)
# user_defined_km_slabs.remove(0)
# temp = temp.dropna()
# temp.set_axis(list(map(lambda x: str(x)+' km bin', user_defined_km_slabs)), inplace=True)
# temp_td = temp.rename(columns={'Leg Distance':'# of trips'})
# temp_td

Cab Capacity,4.0,6.0
km brackets,,
"[0, 10)",1979,1648
"[10, 15)",1721,921
"[15, 20)",1510,803
"[20, 25)",1116,562
"[25, 30)",751,287
"[30, 40)",609,284
"[40, 50)",86,31
"[50, 60)",88,24
"[60, 150)",52,18


###### Trips per day

In [179]:
# filtering out the completed trips

temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']

In [180]:
# create a dataframe '_' that maps trips per day against each cab registration

_ = (temp.groupby('Registration')['Trip Id'].count())/(temp.groupby('Registration')['Leg Date'].nunique())

# now adding columns containing info about capacity, vendor and contract against each registration

temp = pd.merge(_.reset_index(), temp.drop_duplicates(subset='Registration')[['Registration',
                                        'Vendor', 'Contract', 'Cab Capacity']], how='inner', copy=False, on='Registration')
temp.rename(columns={0:'trips per day'}, inplace=True)

In [181]:
# breaking into user defined slabs

user_defined_trip_slabs = [0,2,4,6,8]
temp['trip slab'] = pd.cut(temp['trips per day'], bins=user_defined_trip_slabs, right=False)

# final tabular form 

temp_tpd = temp.pivot_table(values='Registration', columns='Cab Capacity', index='trip slab', aggfunc='count')

###### KM per day

In [182]:
# trip km per day


temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']

# create a dataframe '_' that maps km per day against each cab registration

_ = (temp.groupby('Registration')['Leg Distance'].sum())/(temp.groupby('Registration')['Leg Date'].nunique())

# now adding columns containing info about capacity, vendor and contract against each registration

temp1 = pd.merge(_.reset_index(), temp.drop_duplicates(subset='Registration')[['Registration',
                                        'Vendor', 'Contract', 'Cab Capacity']], how='inner', copy=False, on='Registration')
temp1.rename(columns={0:'trip kms per day'}, inplace=True)

# final table
temp_tkmpd = temp1.pivot_table(values='trip kms per day', index=['Contract', 'Vendor'], columns='Cab Capacity', aggfunc='mean')

In [183]:
# empty leg kms per day


# takingonly rows where empty distance is captured

temp = df_rbd.dropna(subset=['Leg Distance'])

# calculating empty km per day registration wise

_ = temp
_ = (_[_['Leg Type']=='Empty Leg'].groupby('Registration')['Leg Distance'].sum())/(_[_['Leg Type']=='Empty Leg'].groupby(
                                                'Registration')['Leg Date'].nunique())

# merging the frames

temp2 = pd.merge(_.reset_index(), temp.drop_duplicates(subset='Registration')[['Registration', 
                                            'Vendor', 'Contract', 'Cab Capacity']], how='inner', copy=False, on='Registration')
temp2.rename(columns={0:'empty kms per day'}, inplace=True)

# final table
temp_ekmpd = temp2.pivot_table(values='empty kms per day', index=['Contract', 'Vendor'], columns='Cab Capacity', aggfunc='mean')

In [184]:
# total kms per day

temp = temp1.merge(temp2, how='left', on='Registration', copy=False)
temp['empty kms per day'].fillna(0, inplace=True)
temp['total kms'] = temp['trip kms per day'] + temp['empty kms per day']

del [temp1, temp2]

temp_totkmpd = temp.pivot_table(values='total kms', index=['Contract_x', 'Vendor_x'], columns='Cab Capacity_x', aggfunc='mean')

###### Duty hours

In [185]:
print(datetime.now())
temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']
temp['timedelta'] = temp['Duty End'] - temp['Duty Start'] 
temp.drop_duplicates(subset=['Registration', 'Duty Start', 'Duty End'], inplace=True)

def dutyhour(x):
    duty_sum = temp[temp['Registration']==x]['timedelta'].sum()
    duty_sum = duty_sum.days*86400 + duty_sum.seconds
    no_of_days = temp[temp['Registration']==x]['Leg Date'].nunique()
    return (duty_sum/no_of_days)/3600

temp['duty hours'] = temp['Registration'].apply(dutyhour)

temp.drop_duplicates(subset='Registration', inplace=True)
temp_dh = temp.pivot_table(values='duty hours', index=['Contract', 'Vendor'], columns='Cab Capacity', aggfunc='mean')
print(datetime.now())

2020-11-03 12:44:30.698815


C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


2020-11-03 12:44:41.607661


C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###### driving hours

In [103]:
# trip time per day

def timeonly(x):
    format = '%H:%M'
    if x=='':
        return np.nan
    else:
        return datetime.strptime(x, format)

temp = df_rbd[df_rbd['Trip Status']=='Trip Completed']
temp['Leg Start'] = temp['Leg Start'].replace(np.nan, '').apply(timeonly)
temp['Leg End'] = temp['Leg End'].replace(np.nan, '').apply(timeonly)

def triptime(x,y):
    if (y-x)>timedelta(0,0,0):
        return ((y-x).seconds)/60
    else:
        return (((y+timedelta(1,0,0))-x).seconds)/60
temp['timedelta'] = temp.apply(lambda x: triptime(x['Leg Start'], x['Leg End']), axis=1)

def pday(x):
    return temp[temp['Registration']==x]['timedelta'].sum()/temp[temp['Registration']==x]['Leg Date'].nunique()
temp['trip time pd'] = temp['Registration'].apply(pday)


C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Sparsh Agarwal\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [104]:
pd.DataFrame(temp.groupby(['Contract', 'Vendor', 'Leg Date'])['timedelta'].mean()).groupby(['Contract', 'Vendor']).mean()

timedelta
Contract Vendor                    
MSE      Aaron Travels    56.979654
         New Way Travels  57.028423
         Om Sai Travels   56.608025
         Select Cabs      55.860673
MSX      Aaron Travels    43.800387
         New Way Travels  12.497147
         Om Sai Travels   52.801524

## Commercial Optimization

###### number of trips of each contract

In [186]:
# filtering the trips to be included first

temp = df_rbd[df_rbd['Trip Status'].apply(lambda x: True if x in trips_to_include else False)]
temp_tc = pd.DataFrame(temp.groupby('Contract')['Trip Id'].count()).rename(columns={'Trip Id':'# of trips'})

###### number of cabs against vendor and contract type

In [187]:
# taking desirable trips
temp = df_rbd[(df_rbd['Leg Date']>=commercial_start_date) & (df_rbd['Leg Date']<=commercial_end_date)]
temp = temp[temp['Trip Status'].apply(lambda x: True if x in trips_to_include else False)]

#final table, taking unique cabs
temp_cv = temp.pivot_table(values='Registration', index='Vendor', columns='Contract', aggfunc=lambda x: len(x.unique()))

## No-show, Adhoc and escort trips

###### no-show

In [188]:
# shift wise no show%

temp = ((df_ba.groupby('Shift')['No Show'].value_counts())/(df_ba.groupby('Shift')['No Show'].count()))

temp = temp.unstack().loc[temp.unstack().index.drop(labels = ['Login Adhoc', 'Logout Adhoc'])]['Yes']*100
                                                             #'Login Non Shift', 'Logout Non Shift'])]['Yes']*100
temp_nss = pd.DataFrame(temp)
temp_nss = temp_nss.rename(columns={'Yes':'% no show'})

In [189]:
# weekday wise no show

temp = ((df_ba.groupby('Weekday')['No Show'].value_counts())/(df_ba.groupby('Weekday')['No Show'].count()))

temp_nsw = pd.DataFrame(temp.unstack()['Yes']*100)
temp_nsw = temp_nsw.rename(columns={'Yes':'% no show'})

###### adhoc

In [190]:
# shift wise

temp = df_ba[df_ba['Team']!='Escort']
temp = ((temp.groupby('Shift')['SignIn Type'].value_counts())/(temp.groupby('Shift')['SignIn Type'].count()))

temp_ads = pd.DataFrame(temp.unstack()['Adhoc']*100)
temp_ads = temp_ads.rename(columns={'Adhoc':'% of adhocs'})

In [191]:
# weekday wise

temp = df_ba[df_ba['Team']!='Escort']
temp = ((temp.groupby('Weekday')['SignIn Type'].value_counts())/(temp.groupby('Weekday')['SignIn Type'].count()))

temp_adw = pd.DataFrame(temp.unstack()['Adhoc']*100)
temp_adw = temp_adw.rename(columns={'Adhoc':'% of adhocs'})       

###### escort

In [192]:
# shift wise distribution

temp = ((df_ba[df_ba['Team']=='Escort'].pivot_table(values='Trip ID', index='Shift', aggfunc=lambda x: x.nunique()))/(
    df_ba[df_ba['Employee Status']=='Boarded'].pivot_table(values='Trip ID', index='Shift', aggfunc=lambda x: x.nunique()))).dropna()
temp_ess = (temp*100).rename(columns={'Trip ID':'% of escort trips'})

In [193]:
# weekday wise distribution

temp = ((df_ba[df_ba['Team']=='Escort'].pivot_table(values='Trip ID', index='Weekday', aggfunc=lambda x: x.nunique()))/(
    df_ba[df_ba['Employee Status']=='Boarded'].pivot_table(values='Trip ID', index='Weekday', aggfunc=lambda x: x.nunique()))).dropna()
temp_esw = (temp*100).rename(columns={'Trip ID':'% of escort trips'})

In [39]:
# notional escort savings

user_defined_deviation_limit = 2

# taking info of all the escort trips only logout and login wise

#escort = [df_ba.loc[y, 'Trip ID'] for y in list(df_ba.index) if df_ba.loc[y, 'Team']=='Escort']
temp = df_ba[df_ba['Trip ID'].apply(lambda x: x in escort)]
temp = temp[(temp['Employee Status']=='Boarded') & (temp['Team']!='Escort')]
y_out = temp[temp['Direction']=='Logout'].copy()
y_in = temp[temp['Direction']=='Login'].copy()

# working on the logout trips

y_out = y_out.join(y_out.shift(1), rsuffix='_lag', how='left')
y_out = y_out[(y_out['Trip ID']!=y_out['Trip ID'].shift(1)) | (y_out['Trip ID']!=y_out['Trip ID'].shift(2))]
# first male
print(sum((y_out['Trip ID']!=y_out['Trip ID_lag']) & (y_out['Gender']=='Male')))
# distance difference of buddy less than 0
print(sum(y_out[(y_out['Trip ID']==y_out['Trip ID_lag']) & (y_out['Gender']=='Male') & 
          (y_out['Gender_lag']=='Female')]['Distance Travelled(KM)'] >
    y_out[(y_out['Trip ID']==y_out['Trip ID_lag']) & (y_out['Gender']=='Male') & 
          (y_out['Gender_lag']=='Female')]['Distance Travelled(KM)_lag']))
# possible savings based on the user defined limit
print(sum((y_out[(y_out['Trip ID']==y_out['Trip ID_lag']) & (y_out['Gender']=='Male') & 
          (y_out['Gender_lag']=='Female')]['Distance Travelled(KM)_lag'] -
    y_out[(y_out['Trip ID']==y_out['Trip ID_lag']) & (y_out['Gender']=='Male') & 
          (y_out['Gender_lag']=='Female')]['Distance Travelled(KM)']) <= user_defined_deviation_limit/2))

NameError: name 'escort' is not defined

## Exporting files

In [194]:
with pd.ExcelWriter(output_path) as writer:
    temp_su.to_excel(writer, sheet_name='Fleet Mix', startrow=2)
    temp_dev.to_excel(writer, sheet_name='Fleet Mix', startrow=2, startcol=len(temp_su.columns)+4)
    temp_fm.to_excel(writer, sheet_name='Fleet Mix', startrow=2, startcol=len(temp_su.columns)+len(temp_dev.columns)+8)
    temp_td.to_excel(writer, sheet_name='Bill Model', startrow=2)
    temp_tpd.to_excel(writer, sheet_name='Bill Model', startrow=2, startcol=len(temp_td.columns)+4)
    temp_tkmpd.to_excel(writer, sheet_name='Bill Model', startrow=2, startcol=len(temp_td.columns)+len(temp_tpd.columns)+8)
    temp_ekmpd.to_excel(writer, sheet_name='Bill Model', startrow=2, startcol=len(temp_td.columns)+len(temp_tpd.columns)+len(temp_tkmpd.columns)+12)
    temp_totkmpd.to_excel(writer, sheet_name='Bill Model', startrow=2, startcol=len(temp_td.columns)+len(temp_tpd.columns)+len(temp_tkmpd.columns)+len(temp_ekmpd.columns)+16)
    temp_dh.to_excel(writer, sheet_name='Bill Model', startrow=2, startcol=len(temp_td.columns)+len(temp_tpd.columns)+len(temp_tkmpd.columns)+len(temp_ekmpd.columns)+len(temp_totkmpd.columns)+20)
    temp_tc.to_excel(writer, sheet_name='Commercial', startrow=2)
    temp_cv.to_excel(writer, sheet_name='Commercial', startrow=2, startcol=len(temp_tc.columns)+4)
    temp_nss.to_excel(writer, sheet_name='NoShow', startrow=2)
    temp_nsw.to_excel(writer, sheet_name='NoShow', startrow=2, startcol=len(temp_nss.columns)+4)
    temp_ads.to_excel(writer, sheet_name='Adhoc', startrow=2)
    temp_adw.to_excel(writer, sheet_name='Adhoc', startrow=2, startcol=len(temp_ads.columns)+4)
    temp_ess.to_excel(writer, sheet_name='Escort', startrow=2)
    temp_esw.to_excel(writer, sheet_name='Escort', startrow=2, startcol=len(temp_ess.columns)+4)

In [16]:
x = datetime.strptime("12-02-2020 23:45", "%d-%m-%Y %H:%M")
y = datetime.strptime("13-02-2020 00:15", "%d-%m-%Y %H:%M")
a = y-x
a<timedelta(0,0,0)

False

In [20]:
(y-x).seconds

1800

In [ ]:
a = a + datetime.timedelta(1, 1)

In [ ]:
a.days%2

In [ ]:
int(266154/86400)

In [ ]:
p = datetime.timedelta(0, 86399)
q = datetime.timedelta(1, 51)
z = p+q
z = z.days*86400 + z.seconds

In [11]:
[0,1]*3

[0, 1, 0, 1, 0, 1]

In [3]:
workbook = xlsxwriter.Workbook('D:/solutioning/amz-hyd/hello.xlsx')
worksheet = workbook.add_worksheet('Sheet exp')
worksheet.write('A1', 'Hello World')
workbook.close()

In [12]:
q = pd.DataFrame({'col1':['mix', 'mix', 'mix', 'rag', 'rag', 'rag'], 'col2':[0,1]*3, 'col3':[11,12,13,14,15,16]})
q

,col1,col2,col3
0,mix,0,11
1,mix,1,12
2,mix,0,13
3,rag,1,14
4,rag,0,15
5,rag,1,16


In [13]:
q.drop_duplicates(subset='col1')

,col1,col2,col3
0,mix,0,11
3,rag,1,14


In [14]:
q.drop_duplicates(subset=['col1', 'col2'])

,col1,col2,col3
0,mix,0,11
1,mix,1,12
3,rag,1,14
4,rag,0,15


In [15]:
q.drop_duplicates(subset=['col2', 'col1'])

,col1,col2,col3
0,mix,0,11
1,mix,1,12
3,rag,1,14
4,rag,0,15


In [20]:
print(datetime.now())

2020-09-30 17:53:39.019676


In [48]:
temp = df_rbd[df_rbd['Trip Status'].apply(lambda x: x in trips_to_include)]
temp.groupby('Weekday')['Planned Trip Employees'].sum()/temp.groupby('Weekday')['Leg Date'].nunique()

Weekday
Fri    1796.00
Mon    2187.50
Sat      24.00
Sun       0.00
Thu    2462.00
Tue    2472.50
Wed    2430.25
dtype: float64

In [50]:
temp = df_rbd[df_rbd['Trip Status'].apply(lambda x: x in trips_to_include)]
temp = temp.groupby('Leg Date')['Planned Trip Employees'].sum()
print(len(temp[temp>=0.3*temp.max()]))
temp

19


Leg Date
2020-02-01       6.0
2020-02-03    2200.0
2020-02-04    2516.0
2020-02-05    2480.0
2020-02-06    2466.0
2020-02-07    2253.0
2020-02-08      35.0
2020-02-09       0.0
2020-02-10    2194.0
2020-02-11    2467.0
2020-02-12    2222.0
2020-02-13    2477.0
2020-02-14    2128.0
2020-02-15      18.0
2020-02-16       0.0
2020-02-17    2065.0
2020-02-18    2473.0
2020-02-19    2539.0
2020-02-20    2353.0
2020-02-21     497.0
2020-02-22      18.0
2020-02-23       0.0
2020-02-24    2291.0
2020-02-25    2434.0
2020-02-26    2480.0
2020-02-27    2552.0
2020-02-28    2306.0
2020-02-29      43.0
Name: Planned Trip Employees, dtype: float64